In [30]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import seaborn as sns
from warnings import filterwarnings
filterwarnings('ignore')

df = pd.read_csv('lithium-ion batteries.csv')

for i in df.columns:
    print(i, df[i].nunique())

Materials Id 339
Formula 114
Spacegroup 44
Formation Energy (eV) 251
E Above Hull (eV) 108
Band Gap (eV) 307
Nsites 49
Density (gm/cc) 300
Volume 339
Has Bandstructure 2
Crystal System 3


In [31]:
df['Has Bandstructure'] = np.where(df['Has Bandstructure'] == True, 1, 0)
df['Nsites'] = df['Nsites'].astype('float64')


In [32]:
# # split formula into elements if new element starts with capital letter
# df['Formula'] = df['Formula'].str.replace('([A-Z])', r' \1').str.strip().str.split(' ')

# # one hot encoding Formula to their count and multiply
# df = df.join(pd.get_dummies(df['Formula'].apply(pd.Series).stack()).sum(level=0))

# find count of elements in formula
# df['Formula'] = df['Formula'].str.replace('([A-Z])', r' \1').str.strip().str.split(' ')
# df['Formula'] = df['Formula'].apply(lambda x: len(x))
# df['Formula']

df['Crystal System'].value_counts()

monoclinic      139
orthorhombic    128
triclinic        72
Name: Crystal System, dtype: int64

In [33]:
def sigmoid(x):
    return 1/(1 + np.exp(-x))

def dersigmoid(x):
    return x * (1.0 - x)

def loss_func(x, y):
    return (x - y) ** 2

def relu(x):
    return np.maximum(0,x)

def derrelu(x):
    return np.where(x>0, 1, 0)

def softmax(x):
    return np.exp(x)/np.sum(np.exp(x), axis=0)

def dersoftmax(x):
    return softmax(x) * (1 - softmax(x))

def dropout(x, p):
    increase = 1/(1-p)
    return np.where(np.random.rand(*x.shape) > p, x, 0) * increase

def l2_reg(x, lam):
    return lam * np.sum(x**2)

In [47]:
features = df.drop(columns=['Materials Id', 'Formula', 'Spacegroup', 'Crystal System'], axis=1)
target = np.array(pd.get_dummies(df['Crystal System']).astype('float64'))

features = np.array(features)
features = StandardScaler().fit_transform(features)

xtrain, xtest, ytrain, ytest = train_test_split(features, target, random_state=2, test_size=0.2, stratify=target)

In [48]:
def train(xtrain, ytrain, epochs):

    w1 = np.random.random((features.shape[1], 12)) - 0.5
    w2 = np.random.random((12, 24)) - 0.5
    w3 = np.random.random((24, 3)) - 0.5

    for epoch in range(epochs):
        errors = 0
        for i in range(len(xtrain)):
            ytarget = ytrain[i: i+1]

            inpt = xtrain[i: i+1]
            layer1 = sigmoid(np.dot(inpt, w1))
            layer2 = sigmoid(np.dot(layer1, w2))
            outpt = sigmoid(np.dot(layer2, w3))

            error = np.sum(loss_func(outpt, ytarget))
            errors += error

            # back prop
            delta3 = (ytarget - outpt) * dersigmoid(outpt)
            delta2 = delta3.dot(w3.T) * dersigmoid(layer2)
            delta1 = delta2.dot(w2.T) * dersigmoid(layer1)

            w3 += layer2.T.dot(delta3)
            w2 += layer1.T.dot(delta2)
            w1 += inpt.T.dot(delta1)

            dropout(layer1, 0.2)
            dropout(layer2, 0.2)

        
    return w1, w2, w3


In [49]:
w1, w2, w3 = train(xtrain, ytrain, 100)

# train via cross validation
# def cross_val(xtrain, ytrain, epochs):
#     errors = []
#     for i in range(5):
#         xtrain, xtest, ytrain, ytest = train_test_split(xtrain, ytrain, random_state=2, test_size=0.2)
#         w1, w2, w3 = train(xtrain, ytrain, epochs)
#         errors.append(test(xtest, ytest, w1, w2, w3))
#     return np.mean(errors)

# print(cross_val(xtrain, ytrain, 250))


In [50]:
def predict(xtest, w1, w2, w3):
    layer1 = sigmoid(np.dot(xtest, w1))
    layer2 = sigmoid(np.dot(layer1, w2))
    outpt = sigmoid(np.dot(layer2, w3))
    return np.argmax(outpt)

In [51]:
def accuracy(ytest, ypred):
    correct = 0
    for i in range(len(ytest)):
        if ytest[i] == ypred[i]:
            correct += 1
    return correct / len(ytest)

In [52]:
results = []

for i in range(len(xtest)):
    prediction = predict(xtest[i: i+1], w1, w2, w3)
    results.append(prediction)

y_tests = []
ytest = list(ytest)
for i in range(len(ytest)):
    y_tests.append(np.argmax(ytest[i]))   

accuracy(y_tests, results)

0.5

In [53]:
results = []

for i in range(len(xtrain)):
    prediction = predict(xtrain[i: i+1], w1, w2, w3)
    results.append(prediction)

y_tests = []
ytest = list(ytrain)
for i in range(len(ytrain)):
    y_tests.append(np.argmax(ytrain[i]))  

accuracy(y_tests, results) 

0.8265682656826568

In [12]:
def confusion_matrix(ytest, ypred):
    matrix = np.zeros((3,3))
    for i in range(len(ytest)):
        if ytest[i] == ypred[i]:
            if ytest[i] == 0:
                matrix[0][0] += 1
            else:
                matrix[1][1] += 1
        else:
            if ytest[i] == 0:
                matrix[0][1] += 1
            else:
                matrix[1][0] += 1
    return matrix



In [13]:
confusion_matrix(y_tests, results)

array([[101.,  13.,   0.],
       [ 15., 142.,   0.],
       [  0.,   0.,   0.]])